# Draw map and gif

### base imports and tuning

In [17]:
import matplotlib.pyplot as plt
from scipy.ndimage.filters import uniform_filter1d

plt.rcParams['figure.figsize'] = [15, 5]
EPS = 0.000001

In [18]:
import sys
import os

search_tools_path = os.path.join(os.getcwd(), 'search_tools')
if search_tools_path not in sys.path:
    sys.path.append(search_tools_path)

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### pipeline imports

In [20]:
from run_result import EnrichedRunResult
from moving_ai.mai_map_visualizer import VisualizeMaiMap

from moving_ai.mai_map_reader import MaiReader, MaiMaps
from pipeline import *
from run_result import RunResult

from algorithms.astar import Astar
from algorithms.rstar import Rstar
from algorithms.wastar import WAstar
from algorithms.astar_heuristics import diagonal_dist

TypeError: Callable[args, result]: args must be a list. Got <class 'area.Area'>

### draw map pipeline

In [ ]:
def init_map_reader():
    map_reader_0 = MaiReader(MaiMaps.DEN001D)
    selected_task_callback = lambda t: t.start_c == (91, 23) and t.goal_c == (141, 25)
    selected_task = filter(selected_task_callback, map_reader_0.read_tasks()).__next__()
    return map_reader_0.with_task(selected_task)

map_reader = init_map_reader()

In [ ]:
from moving_ai.mai_map import DETAILS, STRIGHT_DELTAS, DIAG_DELTAS, with_move
from random import uniform
from math import pi, cos, sin, sqrt
class RstarWrapper(Area):
    def __init__(self, area: Area):
        assert type(area) == "<class 'moving_ai.mai_map.MapMAI"
        self.area = area

    def dist(self, s1, s2):
        return  sqrt((s1[0]-s2[0])**2+(s1[1]-s2[1])**2)**2
    def get_neighbors(self, s, K, D):
        neighbors = []
        while len(neighbors) < K:
            alpha = uniform(0, 2*pi)
            x, y = int(D*sin(alpha)), int(D*cos(alpha))
            if self.area.traversable((x,y)):
                neighbors.append((x,y))
        return neighbors

In [ ]:
p = Pipeline(
    reader=map_reader,
    algorithms=[Rstar(heuristic=diagonal_dist, w=3, K=10, D=60)],
    res_builder=EnrichedRunResult.create,
    map_wrapper = RstarWrapper,
    processor=VisualizeMaiMap()
)

In [ ]:
p.run()

### draw map gif pipeline

In [ ]:
from moving_ai.mai_map_visualizer import VisualizeMaiMapGif

In [ ]:
p2 = Pipeline(
    reader=map_reader,
    algorithms=[WAstar(heuristic=diagonal_dist, weight=3.)],
    res_builder=EnrichedRunResult.create,
    processor=VisualizeMaiMapGif()
)

In [ ]:
p2.run()
